任务：主 agent 接受来自用户的任务，首先确定问题是否能够回答（护栏边界内），如果在护栏内，那么调用对应的 agent 回答

In [ ]:
from agents.extensions.models.litellm_model import LitellmModel
from agents import set_tracing_disabled
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('DEEPSEEK_API_KEY')
base_url = "https://api.deepseek.com/v1"
chat_model = "deepseek/deepseek-chat"

# chat_model = "mistral-small-latest"
# base_url="https://api.mistral.ai/v1"
# api_key=os.getenv('mistral_key')

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

In [ ]:
from agents import Agent

# 历史
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

# 数学
math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

In [ ]:
from agents import GuardrailFunctionOutput, InputGuardrail, Agent, Runner
from pydantic import BaseModel

class HomeworkOutput(BaseModel):
    """定义输出数据模型"""
    is_homework: bool
    reasoning: str

# 创建护栏代理
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="""Check if the user is asking about homework.
    
    Return your response in the following JSON format:
    {
        "is_homework": true/false,
        "reasoning": "Your reasoning here"
    }
    """,
    output_type=HomeworkOutput,  # 约定输出格式
    model=llm,
)

# 异步函数
async def homework_guardrail(ctx, agent, input_data):
    # 传入护栏代理、输入数据、上下文
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    # 将结果转换为 HomeworkOutput 格式
    return GuardrailFunctionOutput(
        output_info=final_output,  # 包含结果的详细信息
        tripwire_triggered=not final_output.is_homework,  # 出发条件，如果 is_homework=False，则出发护栏
    )

In [ ]:
# 基于用户问题，选择合适的 agent
# triage_agent = Agent(
#     name="Triage Agent",
#     instructions="You determine which agent to use based on the user's homework question",
#     handoffs=[history_tutor_agent, math_tutor_agent],
#     model=llm,
# )

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    # 允许代理将特定任务委派给其他代理
    handoffs=[history_tutor_agent, math_tutor_agent],
    # 可以对输入到代理的内容进行验证
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
    model=llm,
)

In [ ]:
from agents import Runner
from agents.exceptions import InputGuardrailTripwireTriggered

# 测试数学问题（应该通过护栏）
try:
    result = await Runner.run(triage_agent, "欧拉公式的应用？")
    print("结果:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("护栏触发：你的问题不是作业相关")

print("\n" + "="*50 + "\n")

# 测试历史问题（应该通过护栏）
try:
    result = await Runner.run(triage_agent, "美国第一任总统是谁？")
    print("结果:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("护栏触发：你的问题不是作业相关")

print("\n" + "="*50 + "\n")

# 测试非学术问题（应该触发护栏）
try:
    result = await Runner.run(triage_agent, "今天天气怎么样？")
    print("结果:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("护栏触发：你的问题不是作业相关")

In [ ]:
from agents import Runner
import asyncio
from agents.exceptions import InputGuardrailTripwireTriggered

# async def main():
#     result = await Runner.run(triage_agent, "does drinking tee good for the body?")
#     print(result.final_output)

    
# if __name__ == "__main__":
    # asyncio.run(main())

result = await Runner.run(triage_agent, "欧拉公式的应用？")
print(result.final_output)


# try:
#     result = await Runner.run(triage_agent, "who was the first president of the united states?")
#     print(result.final_output)
# except InputGuardrailTripwireTriggered as e:
#     print("你的问题不是作业相关")